# Setup

In [11]:
# Hardware-aware device selection
import torch

def get_device() -> str:
    """
    Returns the optimal device string for running PyTorch models, prioritizing CUDA (NVIDIA GPUs),
    Apple Silicon MPS (Metal), and falling back to CPU. Optionally checks for AMD ROCm/HIP.

    Returns
    ----------
    - device: str
        Name of the device to use: "cuda", "mps", "cpu", or "hip".

    Notes
    ----------
    - On NVIDIA GPUs, "cuda" is used for hardware acceleration.
    - On Apple Silicon, "mps" uses Metal Performance Shaders, providing hardware acceleration.
    - On AMD, "hip" (ROCm) may be available, but this is uncommon.
    - If no GPU is available, it defaults to "cpu".
    Raises
    ----------
    - None
    """
    # Check for CUDA (NVIDIA GPUs)
    if torch.cuda.is_available():
        return "cuda"
    # or MPS (Apple Silicon)
    if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        return "mps"
    # or ROCm (AMD)
    if hasattr(torch.version, "hip") and torch.version.hip is not None:
        return "hip"
    # Fallback to CPU
    return "cpu"

device = get_device()
print(f"Using device: {device}")


Using device: cuda


# Building the emotionRegressor

This class loads both transformers, runs inference, and outputs the 35d mood vector for a single input string. Handles device placement and memory cleanup.


In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np

class EmotionRegressor:
    """
    Loads and manages two transformer models for emotional analysis, producing a 35-dimensional mood vector for a given text input.
    
    Models:
    ----------
    - SamLowe/roberta-base-go_emotions: Predicts confidence scores for 28 discrete emotions (GoEmotions).
    - j-hartmann/emotion-english-roberta-large: Predicts confidence scores for 7 Ekman emotion categories.

    Parameters
    ----------
    - individual_emotion_model_name: str
        HuggingFace model name for the 28d emotion regressor (default: "SamLowe/roberta-base-go_emotions").
    - ekman_category_model_name: str
        HuggingFace model name for the 7d Ekman regressor (default: "j-hartmann/emotion-english-roberta-large").
    - device: str or None
        Device to use for inference ("cuda", "mps", "cpu", etc). If None, selects best available.

    Methods
    -------
    - predict(text: str) -> np.ndarray
        Returns a 35d mood vector for a given text.
    - cleanup() -> None
        Frees model memory for safe re-initialization.
    """
    def __init__(self,
                 individual_emotion_model_name: str = "SamLowe/roberta-base-go_emotions",
                 ekman_category_model_name: str = "j-hartmann/emotion-english-roberta-large",
                 device: str = None):
        """
        Initialize the EmotionRegressor and load both models on the appropriate device.

        Parameters
        ----------
        - individual_emotion_model_name: str
            Name or path for the 28-emotion model.
        - ekman_category_model_name: str
            Name or path for the 7-Ekman-category model.
        - device: str or None
            Device for inference.

        Returns
        -------
        - None

        Raises
        -------
        - RuntimeError: if model loading fails or device is invalid.
        """
        # Load SamLowe model for individual emotions
        self.device = device or get_device()
        self.individual_emotion_tokenizer = AutoTokenizer.from_pretrained(individual_emotion_model_name)
        self.individual_emotion_model = AutoModelForSequenceClassification.from_pretrained(individual_emotion_model_name).to(self.device)
        self.individual_emotion_model.eval()
        
        # Load hartmann model for Ekman categories
        self.ekman_category_tokenizer = AutoTokenizer.from_pretrained(ekman_category_model_name)
        self.ekman_category_model = AutoModelForSequenceClassification.from_pretrained(ekman_category_model_name).to(self.device)
        self.ekman_category_model.eval()

    def predict(self, text: str) -> np.ndarray:
        """
        Generates a 35-dimensional mood vector for the given input text.

        Parameters
        ----------
        - text: str
            The text to analyze for emotion content.

        Returns
        -------
        - mood_vector: np.ndarray, shape (35,)
            Concatenated vector: [28d GoEmotions scores] + [7d Ekman scores]

        Raises
        -------
        - ValueError: if the text input is empty or not a string.
        - RuntimeError: if inference fails due to hardware or model issues.
        """
        # Validate input
        if not isinstance(text, str) or not text.strip():
            raise ValueError("Input text must be a non-empty string.")
        
        # Lowe model output (28d)
        inputs = self.individual_emotion_tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(self.device)
        with torch.no_grad():
            individual_emotion_logits = self.individual_emotion_model(**inputs).logits
            individual_emotion_probs = torch.sigmoid(individual_emotion_logits).cpu().numpy().flatten()  # (28,)
            
        # Hartmann model output (7d)
        inputs = self.ekman_category_tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(self.device)
        with torch.no_grad():
            ekman_category_logits = self.ekman_category_model(**inputs).logits
            ekman_category_probs = torch.sigmoid(ekman_category_logits).cpu().numpy().flatten()  # (7,)
            
        # Combine into 35d vector
        mood_vector = np.concatenate([individual_emotion_probs, ekman_category_probs])
        return mood_vector

    def cleanup(self) -> None:
        """
        Releases GPU/CPU memory by deleting model weights.

        Returns
        -------
        - None
        """
        # Delete models to free memory
        del self.individual_emotion_model, self.ekman_category_model
        import gc
        # Run garbage collection to free up memory
        gc.collect()
        # Clear CUDA cache if using GPU
        if self.device == "cuda":
            import torch
            torch.cuda.empty_cache()


# Demo of 35-dimensional mood_vector

## Hard-coded example

In [13]:
# Initialize and demo
emotion_regressor = EmotionRegressor(device=device)
sample_text = "I'm exhausted and nothing feels worth it. I just want to sleep and not think about school."
mood_vector = emotion_regressor.predict(sample_text)
print("35d mood_vector:", mood_vector)
print("Shape:", mood_vector.shape)
emotion_regressor.cleanup()


35d mood_vector: [0.00326003 0.00309266 0.00642972 0.06645703 0.01301786 0.00869487
 0.00228629 0.0040405  0.56660604 0.21256632 0.02513171 0.01049309
 0.00295363 0.00560902 0.00179183 0.00208721 0.00450544 0.01198611
 0.00578792 0.00637366 0.01292751 0.00123158 0.01425911 0.00374253
 0.00773546 0.18360841 0.00220226 0.07846184 0.40809634 0.24985953
 0.2786624  0.21017288 0.6955236  0.973397   0.29766086]
Shape: (35,)


## Live-user example (doesn't work)

In [14]:
# while True:
#     user_input = input("Enter text (or 'quit'): ")
#     if user_input.lower() == "quit":
#         break
#     mood_vector = emotion_regressor.predict(user_input)
#     print("35d mood_vector:", mood_vector)


## Tinkering with output display format

In [15]:
import pandas as pd

# These should be the order of the GoEmotions and Ekman categories used by your models
GOEMOTIONS_LABELS = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment',
    'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness',
    'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'
]
EKMAN_LABELS = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']

def pretty_print_mood_vector(mood_vector):
    """
    Prints the 35d mood vector in a readable format using pandas DataFrame.
    """
    if len(mood_vector) != 35:
        raise ValueError("Mood vector must be length 35")
    data = {**{f"{e}": v for e, v in zip(GOEMOTIONS_LABELS, mood_vector[:28])},
            **{f"ekman_{e}": v for e, v in zip(EKMAN_LABELS, mood_vector[28:])}}
    df = pd.DataFrame([data])
    print(df.T.rename(columns={0: 'confidence'}))  # .T to flip to (label, value) per row

# Usage
pretty_print_mood_vector(mood_vector)


                confidence
admiration        0.003260
amusement         0.003093
anger             0.006430
annoyance         0.066457
approval          0.013018
caring            0.008695
confusion         0.002286
curiosity         0.004041
desire            0.566606
disappointment    0.212566
disapproval       0.025132
disgust           0.010493
embarrassment     0.002954
excitement        0.005609
fear              0.001792
gratitude         0.002087
grief             0.004505
joy               0.011986
love              0.005788
nervousness       0.006374
optimism          0.012928
pride             0.001232
realization       0.014259
relief            0.003743
remorse           0.007735
sadness           0.183608
surprise          0.002202
neutral           0.078462
ekman_anger       0.408096
ekman_disgust     0.249860
ekman_fear        0.278662
ekman_joy         0.210173
ekman_neutral     0.695524
ekman_sadness     0.973397
ekman_surprise    0.297661
